In [1]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
# file = '/home/zzhang/OneDrive/data/High_entropy/script/calibration.npy'
# image = np.load(file)
dx= 0.1645429228960236	
# dx = 0.36254824166962796
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [2]:
model.plot()

In [3]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05)

# model.select_region(invert_selection=False)

Updated peak locations.


array([[1106., 1812.],
       [1130., 1763.],
       [ 207., 1977.],
       ...,
       [1555.,    0.],
       [2043.,    0.],
       [1059.,    0.]])

In [4]:
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'],min_distance=20,a_limit=50, b_limit=50,reciprocal=True)

Origin selected: [1012.  906.]
Vector a selected: [24. -1.]
Vector b selected: [ -2. -27.]
In pixel: Origin: [1012.  906.], Vector a: [24. -1.], Vector b: [ -2. -27.]
In space: Origin: [166.51743797 149.07588814] A, Vector a: [ 3.94903015 -0.16454292] A, Vector b: [-0.32908585 -4.44265892] A
Origin selected: [1024. 1024.]
Vector a selected: [90.9978355   8.36580087]
Vector b selected: [ 0. 84.]
Origin selected: [1024. 1024.]
Vector a selected: [84.  0.]
Vector b selected: [ 0. 84.]
In pixel: Origin: [1024. 1024.], Vector a: [84.  0.], Vector b: [ 0. 84.]
In space: Origin: [3.03872079 3.03872079] 1/A, Vector a: [0.24927006 0.        ] 1/A, Vector b: [0.         0.24927006] 1/A
FFT real a: [24.38095238  0.        ] pixel, Real b: [ 0.         24.38095238] pixel


In [6]:
model.add_or_remove_peaks(min_distance=10)

Updated peak locations.


array([[   9.        ,  164.        ],
       [  10.        ,  115.61904762],
       [  10.        ,  140.        ],
       ...,
       [2042.19047619, 1703.19047619],
       [2042.19047619, 1728.19047619],
       [2042.19047619, 1778.19047619]])

In [5]:
model.select_region()

INFO:root:Selected 13185 atoms out of 14083 atoms.


In [7]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=5)

Fitting random batch:   0%|          | 0/14 [00:00<?, ?it/s]INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': Found CUDA version 12010, but JAX was built against version 12020, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built.
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
Fitting random batch:   0%|          | 0/14 [00:00<?, ?it/s]


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory allocating 228891557888 bytes.

: 

In [20]:
model.fit_random_batch(batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.1,verbose=False,num_epoch=5)

Fitting random batch: 100%|██████████| 14/14 [01:01<00:00,  4.41s/it]
INFO:root:Convergence rate for pos_x = 6.13818359375
INFO:root:Convergence not reached
Fitting random batch: 100%|██████████| 14/14 [00:25<00:00,  1.85s/it]
INFO:root:Convergence rate for pos_x = nan
INFO:root:Convergence rate for pos_y = nan
INFO:root:Convergence rate for height = nan
INFO:root:Convergence rate for sigma = nan
INFO:root:Convergence rate for background = nan
INFO:root:Convergence reached


ValueError: cannot convert float NaN to integer

In [22]:
model.plot_fitting()

In [18]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [19]:
model.plot_scs()

In [1]:
from qem.crystal_analyzer import CrystalAnalyzer
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'
image = np.loadtxt(file)
dx= 0.1645429228960236	
model=ImageModelFitting(image, dx=dx,units='A')
model.find_peaks()
crystal_analyzer = CrystalAnalyzer(image = model.image, dx = model.dx, peak_positions = model.coordinates, atom_types = model.atom_types, elements=['Sr','Ti'])

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Updated peak locations.


INFO:root:No atoms selected.


In [6]:

cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
crystal_analyzer.read_cif(cif_file)

Atoms(symbols='SrTi', pbc=True, cell=[3.905, 3.905, 3.905], spacegroup_kinds=...)

In [4]:
crystal_analyzer.choose_lattice_vectors()

Selection incomplete.
Selection incomplete.


TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'

In [5]:
crystal_analyzer._origin_adaptive = None
shift_center = crystal_analyzer.shift_origin_adaptive(50, 50)
plt.imshow(model.image, cmap='gray')
peaks = []
for key, value in shift_center.items():
    peaks.append(value)

plt.scatter(np.array(peaks)[:,0], np.array(peaks)[:,1], c='r', s=10)
plt.show()

In [6]:
peak_positions, atom_types = crystal_analyzer.generate_supercell_lattice(a_limit=50, b_limit=50)

In [14]:
peak_positions, atom_types = crystal_analyzer.supercell_project_2d()

In [7]:
plt.scatter(peak_positions[:,0], peak_positions[:,1], c='r', s=10)

In [10]:
crystal_analyzer.unitcell_mapping()

TypeError: list indices must be integers or slices, not numpy.float64

In [23]:
im_refined = model.predict_local(params=model.params)
plt.imshow(im_refined)

In [25]:
plt.imshow(model.image-im_refined)

In [18]:
model.atom_types

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,